In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import requests
import bs4
import time
import matplotlib.pyplot as plt
import neo_quant as neo

In [2]:
pd.options.display.max_rows = 60
%matplotlib inline

In [3]:
companies = neo.get_company_info(0)
companies = neo.company_apply_a_prefix(companies)

In [9]:
firm_code = neo.get_company_code(companies, '삼성전자')

In [ ]:
# firm_code = 'A005930'
fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
fs_page = requests.get(fs_url)
fs_tables = pd.read_html(fs_page.text)

temp_df = fs_tables[0]
temp_df = temp_df.set_index(temp_df.columns[0])
# temp_df = temp_df[temp_df.columns[:4]]
# temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

temp_df2 = fs_tables[2]
temp_df2 = temp_df2.set_index(temp_df2.columns[0])
# temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

temp_df3 = fs_tables[4]
temp_df3 = temp_df3.set_index(temp_df3.columns[0])
# temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

fs_df = pd.concat([temp_df, temp_df2, temp_df3])


In [15]:
fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
fr_page = requests.get(fr_url)
fr_tables = pd.read_html(fr_page.text)

fr_df = fr_tables[0]
fr_df = fr_df.set_index(fr_df.columns[0])
# temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
#                        '부채비율계산에 참여한 계정 펼치기',
#                        '영업이익증가율계산에 참여한 계정 펼치기',
#                        'ROA계산에 참여한 계정 펼치기',
#                        'ROIC계산에 참여한 계정 펼치기']]
# temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']


In [17]:
invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
invest_page = requests.get(invest_url)
invest_tables = pd.read_html(invest_page.text)
iv_df = invest_tables[1]

iv_df = iv_df.set_index(iv_df.columns[0])
# temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
#                    'PCR계산에 참여한 계정 펼치기',
#                    'PSR계산에 참여한 계정 펼치기',
#                    'PBR계산에 참여한 계정 펼치기',
#                   '총현금흐름']]
# temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']

In [26]:
iv_df.loc['EPS계산에 참여한 계정 펼치기(원)']

2015/12    2198
2016/12    2735
2017/12    5421
2018/12    6024
2019/06    1498
Name: EPS계산에 참여한 계정 펼치기(원), dtype: object

In [30]:
fs_df.loc[['영업이익','당기순이익']]

,2016/12,2017/12,2018/12,2019/06,전년동기,전년동기(%)
IFRS(연결),,,,,,
영업이익,292407.0,536450.0,588867.0,128303.0,305112.0,-57.9
당기순이익,227261.0,421867.0,443449.0,102242.0,227320.0,-55.0


In [22]:
companies.loc[firm_code]

기업명                 삼성전자
구분                   코스피
액면가(원)               100
상장주식수(주)      5969782550
자본금(원)      897513820000
Name: A005930, dtype: object

In [31]:
from IPython.display import HTML
pd.set_option('display.max_colwidth', -1)

In [35]:
price_path = r'data/price_data.xlsx'
price_df = pd.read_excel(price_path)
price_df = price_df.set_index(price_df.columns[0])

In [37]:
def get_maximum_profit_df(price_df):
    last_price = price_df.iloc[-1]
    first_price = price_df['2016/10'].iloc[0]
    price_diff_df = pd.DataFrame({last_price.name:last_price, first_price.name:first_price})
    price_diff_df.index = 'A' + price_diff_df.index
    price_diff_df['diff'] = price_diff_df[last_price.name] - price_diff_df[first_price.name]
    price_diff_df = price_diff_df[price_diff_df[last_price.name] > 5000]
    price_diff_df = price_diff_df[price_diff_df['diff'] > 0]
    # price_diff_df['ratio'] = price_diff_df['diff'] / price_diff_df[first_price.name]
    price_diff_df['ratio'] = price_diff_df[last_price.name] / price_diff_df[first_price.name]
    price_diff_df = price_diff_df.sort_values(by='ratio', ascending=False)
    # price_diff_df = add_company_info(price_diff_df, companies)
    # price_diff_df = get_kospi_list(price_diff_df)
    price_diff_df['fs_info'] = price_diff_df.index
    price_diff_df['fs_info'] = price_diff_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
    price_diff_df['fr_info'] = price_diff_df.index
    price_diff_df['fr_info'] = price_diff_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
    price_diff_df['iv_info'] = price_diff_df.index
    price_diff_df['iv_info'] = price_diff_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
    return HTML(price_diff_df.to_html(escape=False))


In [52]:
firm_name = 'GS'
firm_code = neo.get_company_code(companies, firm_name)
firm_code = firm_code.replace('A','')
last_price = price_df[firm_code].iloc[-1]
firm_df = pd.DataFrame({'price':[last_price]}, index=[firm_name])
firm_df
# price_diff_df.index = 'A' + price_diff_df.index
# price_diff_df['diff'] = price_diff_df[last_price.name] - price_diff_df[first_price.name]
# price_diff_df = price_diff_df[price_diff_df[last_price.name] > 5000]
# price_diff_df = price_diff_df[price_diff_df['diff'] > 0]
# # price_diff_df['ratio'] = price_diff_df['diff'] / price_diff_df[first_price.name]
# price_diff_df['ratio'] = price_diff_df[last_price.name] / price_diff_df[first_price.name]
# price_diff_df = price_diff_df.sort_values(by='ratio', ascending=False)
# # price_diff_df = add_company_info(price_diff_df, companies)
# # price_diff_df = get_kospi_list(price_diff_df)
firm_df['fs_info'] = firm_df.index
firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
firm_df['fr_info'] = firm_df.index
firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
firm_df['iv_info'] = firm_df.index
firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
HTML(firm_df.to_html(escape=False))

,price,fs_info,fr_info,iv_info
GS,49850,fs,fr,iv


In [54]:
def get_company_info(company_df, price_df, firm_name):
    firm_code = neo.get_company_code(companies, firm_name)
    firm_code = firm_code.replace('A','')
    last_price = price_df[firm_code].iloc[-1]
    firm_df = pd.DataFrame({'price':[last_price]}, index=[firm_name])
    firm_df['fs_info'] = firm_df.index
    firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
    firm_df['fr_info'] = firm_df.index
    firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
    firm_df['iv_info'] = firm_df.index
    firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
    return HTML(firm_df.to_html(escape=False))

get_company_info(companies, price_df, 'GS')

,price,fs_info,fr_info,iv_info
GS,49850,fs,fr,iv


In [83]:
firm_name = 'GS'
firm_code = neo.get_company_code(companies, firm_name)
firm_info = companies.loc[firm_code]
firm_info = firm_info.to_frame(name=firm_name)
firm_info = firm_info.T
# firm_info
# pd.concat(firm_info)
firm_df = firm_info.join(pd.DataFrame({'code':[firm_code]}, index=[firm_name]))
firm_df['fs_info'] = firm_df.index
firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
firm_df['fr_info'] = firm_df.index
firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
firm_df['iv_info'] = firm_df.index
firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
HTML(firm_df.to_html(escape=False))
# firm_info = firm_info['code'] = [firm_code]
# firm_code = firm_code.replace('A','')
# last_price = price_df[firm_code].iloc[-1]
# firm_df = firm_info['price'] = [last_price]

# firm_df = pd.DataFrame({firm_name, firm_info})

,기업명,구분,액면가(원),상장주식수(주),자본금(원),code,fs_info,fr_info,iv_info
GS,GS,코스피,5000,92915378,473501020000,A078930,fs,fr,iv


In [84]:
# firm_name = 'GS'
def get_company_info(company_df, price_df, firm_name):
    firm_code = get_company_code(companies, firm_name)
    firm_info = companies.loc[firm_code]
    firm_info = firm_info.to_frame(name=firm_name)
    firm_info = firm_info.T
    # firm_info
    # pd.concat(firm_info)
    firm_df = firm_info.join(pd.DataFrame({'code':[firm_code]}, index=[firm_name]))
    firm_df['fs_info'] = firm_df.index
    firm_df['fs_info'] = firm_df['fs_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode={0}" target="_blank">fs</a>'.format(x))
    firm_df['fr_info'] = firm_df.index
    firm_df['fr_info'] = firm_df['fr_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode={0}" target="_blank">fr</a>'.format(x))
    firm_df['iv_info'] = firm_df.index
    firm_df['iv_info'] = firm_df['iv_info'].apply(lambda x: '<a href="https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode={0}" target="_blank">iv</a>'.format(x))
    return HTML(firm_df.to_html(escape=False))
# firm_info = firm_info['code'] = [firm_code]
# firm_code = firm_code.replace('A','')
# last_price = price_df[firm_code].iloc[-1]
# firm_df = firm_info['price'] = [last_price]

# firm_df = pd.DataFrame({firm_name, firm_info})
get_company_info(companies, price_df, 'GS')

,기업명,구분,액면가(원),상장주식수(주),자본금(원),code,fs_info,fr_info,iv_info
GS,GS,코스피,5000,92915378,473501020000,A078930,fs,fr,iv
